# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:
from nltk.util import ngrams

In [160]:
import numpy as np
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
import string
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
DATA_DIR='names_data'

In [412]:
# Читаем данные и заполняем списки
females = read_file('female.txt')
males   = read_file('male.txt')
females_uniq = np.setdiff1d(females, males)
males_uniq = np.setdiff1d(males, females)
females_long = []
for i in females_uniq:
    if len(i) < 4:
     #   print(i)
        True
    else:
        females_long.append(i)
        
males_long = []
for i in males_uniq:
    if len(i) < 4:
     #   print(i)
        True
    else:
        males_long.append(i)

In [403]:
TRAIN_SIZE = 2000
TEST_SIZE = 440
NGRAM_SIZE = 3

#m_females = females
#m_males   = males

# Для обучения только на уникальных именах
m_females = females_uniq
m_males   = males_uniq


# Обучение на именах, длиннее 4 символов.
#m_females = females_long
#m_males =   males_long

In [404]:
female_train, female_test = train_test_split(m_females, train_size = TRAIN_SIZE, test_size = TEST_SIZE)
male_train, male_test = train_test_split(m_males, train_size = TRAIN_SIZE, test_size = TEST_SIZE)

X_train_names = np.concatenate((female_train, male_train))
X_test_names = np.concatenate((female_test, male_test))

y_train = np.concatenate((np.zeros(TRAIN_SIZE),np.ones(TRAIN_SIZE))) 
y_test = np.concatenate((np.zeros(TEST_SIZE), np.ones(TEST_SIZE)))

X_train_ngrams = get_ngrams_list(X_train_names,NGRAM_SIZE)
X_test_ngrams = get_ngrams_list(X_test_names, NGRAM_SIZE)

cv = CountVectorizer(analyzer='word')
cv.fit(X_train_ngrams)
X_train = cv.transform(X_train_ngrams)
X_test = cv.transform(X_test_ngrams)

In [405]:
X_train.shape

(4000, 2629)

In [411]:
MNB_clf = MultinomialNB(alpha=2)
MNB_clf.fit(X_train, y_train)
train_predicted = MNB_clf.predict(X_train)
print("TRAIN_Scores")
print_scores(train_predicted, y_train)
print("TEST_Scores")
print_scores(MNB_clf.predict(X_test), y_test)

TRAIN_Scores
acc=0.8662
micro F1=0.8662, micro P=0.8662, micro R=0.4630, micro RC=0.8662
macro F1=0.8661, macro P=0.8662, macro R=0.4630, macro RC=0.8676 

TEST_Scores
acc=0.7795
micro F1=0.7795, micro P=0.7795, micro R=0.1066, micro RC=0.7795
macro F1=0.7788, macro P=0.7795, macro R=0.1066, macro RC=0.7832 



Лучшие показатели для N-грамм из трех символов: на тестовой выборек  accuracy = 0.7864, F1=0.7863 при alpha = 1. <br>
Для N-грамм из 3 символов -  accuracy  = 0.7795, F1 = 0.7788<br>
Для N-грамм из 2 символов -  accuracy  = 0.7307, F1 = 0.7303

# Выведем список ошибочных предсказаний.

In [366]:
test_pred = MNB_clf.predict(X_test)
for i in range (len(y_test)):
    if y_test[i] != test_pred[i]:
        print("name = {}; predicted = {}".format(X_test_names[i], test_pred[i]))

name = Codie; predicted = 1.0
name = Tiffie; predicted = 1.0
name = Esther; predicted = 1.0
name = Hester; predicted = 1.0
name = Kellsie; predicted = 1.0
name = Olive; predicted = 1.0
name = Gera; predicted = 1.0
name = Vilma; predicted = 1.0
name = Jeniffer; predicted = 1.0
name = Neilla; predicted = 1.0
name = Kiele; predicted = 1.0
name = Thia; predicted = 1.0
name = Aili; predicted = 1.0
name = Desdemona; predicted = 1.0
name = Kevina; predicted = 1.0
name = Steffie; predicted = 1.0
name = Andromache; predicted = 1.0
name = Allison; predicted = 1.0
name = Ericka; predicted = 1.0
name = Farand; predicted = 1.0
name = Delphine; predicted = 1.0
name = Micki; predicted = 1.0
name = Sylvie; predicted = 1.0
name = Fortuna; predicted = 1.0
name = Veradis; predicted = 1.0
name = Constancy; predicted = 1.0
name = Cariotta; predicted = 1.0
name = Andromeda; predicted = 1.0
name = Mikako; predicted = 1.0
name = Quintilla; predicted = 1.0
name = Francoise; predicted = 1.0
name = Frances; pred

Источников ошибок, судя по всему, несколько:
 * Некоторые н-грамы встречаются только один раз, например в Lucky, Luck - есть только одна н-грамма в тестовой выборке, в тоже  время ucky - есть среди мужских имен. 
 * Созвучные имена - Emily (ж) и Emile(м). Robb(м) и Robby(ж), Robbi(ж), Robbin (ж), Robbyn(ж). 
 

# Library

In [159]:
def print_scores(pred, true):
    acc = accuracy_score(pred,true)
    acc = accuracy_score(pred, true)
    micro_f1 = f1_score(pred, true, average = 'micro')
    micro_p =  precision_score(pred, true, average = 'micro')
    micro_rc =  recall_score(pred, true, average = 'micro' )
    micro_r =  r2_score(pred, true)
    macro_f1 = f1_score(pred, true, average = 'macro')
    macro_p =  precision_score(pred, true, average = 'macro')
    macro_r =  r2_score(pred, true)
    macro_rc = recall_score(pred, true, average = 'macro' )
    print('acc={0:1.4f}'.format(acc))
    print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}, micro RC={3:1.4f}'.format(micro_f1, micro_p, micro_r, micro_rc))
    print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}, macro RC={3:1.4f} \n'.format(macro_f1, macro_p, macro_r, macro_rc))

In [114]:
def read_file(fn:str):
    """
        Прочитать файл и вернуть np.array(dtype=np.str())
    """
    f = open(DATA_DIR + '/' + fn, 'r')
    t = []
    for i in f:
        i = i.strip()
        t.append(i)
    rv = np.array(t, dtype=np.str)
    return rv

In [115]:
def get_ngrams_list(l: list, n: int):
    a = []
    for i in l:
        t = ngrams(i,n)
        t2 = ''
        for j in t:
            t2 += ''.join((map(str,j))) + ' '
        a.append(t2)
    return a

In [188]:
def remove_dup_names(a: list, b:list):
    for i in a:
        if i in b:
            a.remove(i)
            b.remove(i)

# Flood.

In [120]:
t = get_ngrams_list(['abcdef', 'qwerty'],4)
cv = CountVectorizer(analyzer='word')
cv.fit(t)
cv.vocabulary_

{'abcd': 0, 'bcde': 1, 'cdef': 2, 'erty': 3, 'qwer': 4, 'wert': 5}

In [191]:
t = np.array(['a','b','c','d'])
np.delete(t['a'])
t

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices